In [1]:
import sys
import math
import numpy as np

In [141]:
class Pac(object):
    def __init__(self, x, y, pac_id, type_id, speed_turns_left, ability_cooldown):
        self.x = x
        self.y = y
        self.pac_id = pac_id
        self.type_id = type_id
        self.speed_turns_left = speed_turns_left
        self.ability_cooldown = ability_cooldown
        self.dest = None
    def __eq__(self, another):
        return self.pac_id == another.pac_id
    def __hash__(self):
        return self.pac_id
    def update(self, x, y, type_id, speed_turns_left, ability_cooldown):
        self.type_id = type_id
        self.speed_turns_left = speed_turns_left
        self.ability_cooldown = ability_cooldown
        self.x = x
        self.y = y
    def action(self):
        return ""

class Pacs(object):
    def __init__(self):
        self.pacs = {}
    def update(self, my_pacs):
        for x, y, pac_id, type_id, speed_turns_left, ability_cooldown in my_pacs:
            if pac_id not in self.pacs:
                self.pacs[pac_id] = Pac(x, y, pac_id, type_id, speed_turns_left, ability_cooldown)
            else:
                self.pacs[pac_id].update(x, y, type_id, speed_turns_left, ability_cooldown)
    def action(self):
        return "|".join([p.action() for p in self.pacs.values()])
    
class Grid(object):
    def __init__(self, map_grid):
        # -2 wall, -1 unknown, >=0 pallet value
        self.width = len(map_grid)
        self.height = len(map_grid[0])
        self.map_grid = np.zeros((self.width, self.height))
        for i in range(self.width):
            for j in range(self.height):
                if map_grid[i][j] == "#":
                    self.map_grid[i][j] = -2
                else:
                    self.map_grid[i][j] = -1
        self.my_pacs = Pacs()
        self.oppenent_pacs = Pacs()
    def update_pallets(self, pallets_list):
        my_vision = self.pacs_vision(self.my_pacs)
        self.map_grid[self.map_grid > 0] = -1
        for i in my_vision:
            self.map_grid[i] = 0
        for x, y, value in pallets_list:
            self.map_grid[x][y] = value    
    def pacs_vision(self, pacs):
        visible_points = set()
        for pac in pacs.pacs.values():
            visible_points |= self.get_visible_points(pac)
        return visible_points
    def get_visible_points(self, pac):
        visible_points = set()
        directions = np.array([[-1,0], [1,0], [0,1], [0,-1]])
        for direction in directions:
            pos = (pac.x, pac.y)
            l = max(self.height, self.width)
            while self.map_grid[pos] != -2 and l > 0:
                visible_points.add(pos)
                pos = tuple((pos + direction) % np.array([self.width, self.height]))
                l -= 1
        return visible_points
    def __repr__(self):
        return self.map_grid.__repr__()

In [142]:
map_grid = [[" ", " ", "#", " ", " "],
           ["#", "#", "#", " ", "#"],
           [" ", " ", " ", " ", " "],
           ["#", "#", "#", "#", "#"]]

In [143]:
grid = Grid(map_grid)
p = Pac(0,0,None,None,None, None)
grid.my_pacs.pacs["1"] = p

In [144]:
grid.map_grid

array([[-1., -1., -2., -1., -1.],
       [-2., -2., -2., -1., -2.],
       [-1., -1., -1., -1., -1.],
       [-2., -2., -2., -2., -2.]])

In [145]:
pallets_list = [[0,1,1], [0,3,1], [0,4,10]]

In [146]:
grid.update_pallets(pallets_list)

In [147]:
grid.map_grid

array([[ 0.,  1., -2.,  1., 10.],
       [-2., -2., -2., -1., -2.],
       [-1., -1., -1., -1., -1.],
       [-2., -2., -2., -2., -2.]])